In [ ]:
!pip install -U langchain-community
!pip install gradio transformers torch
!pip install qdrant-client


In [ ]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from qdrant_client import QdrantClient
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
import gradio as gr

In [ ]:
# Cài đặt thông tin truy cập vector database
QDRANT_API_KEY = "..." #Tạo ở Qdrant
QDRANT_URL = "..."
HUGGINGFACE_API_KEY= "..."

In [ ]:
# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Hệ thống hỏi đáp về các văn bản:")
    from transformers import AutoTokenizer, AutoModelForQuestionAnswering
    import torch
    import gradio as gr
    from qdrant_client import QdrantClient
    import numpy as np

    # Khởi tạo mô hình và tokenizer cho QA
    tokenizer = AutoTokenizer.from_pretrained("PhucDanh/Bartpho-fine-tuning-model-for-question-answering")
    model = AutoModelForQuestionAnswering.from_pretrained("PhucDanh/Bartpho-fine-tuning-model-for-question-answering")
    # Delete colection if exist to reup all documents
    client = QdrantClient(
        url=QDRANT_URL, prefer_grpc=False, api_key=QDRANT_API_KEY,
    )

    # Load the embedding model
    embeddings = HuggingFaceInferenceAPIEmbeddings(
        model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
        api_key = HUGGINGFACE_API_KEY,
        #model_kwargs = {'device': 'auto'}
    )
    collection_name = "Data_chatbot"

    def search_from_qdrant(query, top_k=2):
        try:
            # Tạo đối tượng db kết nối với Qdrant và sử dụng embeddings đã được khởi tạo
            db = Qdrant(client=client, embeddings=embeddings, collection_name=collection_name)

            # Tìm kiếm tương đồng với điểm số (kết quả trả về là một danh sách các tài liệu)
            docs = db.similarity_search_with_score(query=query, k=2)

            # for doc, score in docs:  # Unpack the tuple into doc and score
                # print(doc.page_content)  # Access page_content directly from the Document object
                # print(doc.metadata)

            # Lấy nội dung của các tài liệu từ payload
            contexts = [doc.page_content for doc, score in docs]  # Lấy 'page_content'
            return contexts
        except Exception as e:
            print("Lỗi khi tìm kiếm từ Qdrant:", e)
            return []


    # contexts = search_from_qdrant(question)


    # Hàm trả lời câu hỏi với context tìm được từ Qdrant
    def answer_question(question, contexts):
        # Tìm context từ Qdrant
        # contexts = search_from_qdrant(question)
        # context = " ".join(contexts)  # Kết hợp các context lại thành một đoạn văn bản dài

        # Tiến hành trả lời câu hỏi
        inputs = tokenizer(question, contexts, return_tensors="pt")
        if 'token_type_ids' in inputs:
            del inputs['token_type_ids']
        with torch.no_grad():
            outputs = model(**inputs)

        answer_start_index = outputs.start_logits.argmax()
        answer_end_index = outputs.end_logits.argmax()

        predict_answer_tokens = inputs.input_ids[0, answer_start_index: answer_end_index + 1]
        return tokenizer.decode(predict_answer_tokens)
    def process_question(question):
        # Tìm kiếm context từ Qdrant
        contexts = search_from_qdrant(question)
        context = " ".join(contexts)
        if not context:
            return "Không tìm thấy thông tin phù hợp trong dữ liệu."

        # Trả lời câu hỏi dựa trên context
        answer = answer_question(question, context)
        return answer



    # gr.Markdown("# Hệ thống hỏi đáp về các văn bản:")

    with gr.Row():
        txt_question = gr.Textbox(lines=2, label="Câu hỏi")
        btn_answer = gr.Button("Tìm câu trả lời")

    with gr.Row():
        txt_answer = gr.Textbox(label="Câu trả lời:", lines=5)

    # Kết nối chức năng trả lời câu hỏi với nút bấm
    btn_answer.click(fn=process_question, inputs=txt_question, outputs=txt_answer)

demo.launch(share=True)
